### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
!uv pip install langchain

Using Python 3.11.11 environment at: /Users/vaibhavarde/Desktop/AgentKrish/AgentNotes/.venv
Audited 1 package in 53ms


In [4]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

# import openai
# openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
# groq_api_key

In [3]:
!uv pip install langchain_groq

Using Python 3.11.11 environment at: /Users/vaibhavarde/Desktop/AgentKrish/AgentNotes/.venv
Audited 1 package in 9ms


In [8]:
# from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-120b",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x11531b190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x115b0db50>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
!uv pip install langchain_core

Using Python 3.11.11 environment at: /Users/vaibhavarde/Desktop/AgentKrish/AgentNotes/.venv
Audited 1 package in 8ms


In [9]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [10]:
result

AIMessage(content='Bonjour, comment ça va\u202f?', additional_kwargs={'reasoning_content': 'We need to translate "Hello How are you?" to French. The user wants translation. Should output "Bonjour, comment ça va ?" or "Bonjour, comment allez-vous ?" Since informal maybe "Bonjour, comment ça va ?" Provide translation.'}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 86, 'total_tokens': 151, 'completion_time': 0.136717148, 'completion_tokens_details': {'reasoning_tokens': 49}, 'prompt_time': 0.00320056, 'prompt_tokens_details': None, 'queue_time': 0.05067975, 'total_time': 0.139917708}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_fd1fe7f861', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--3e707aa8-79bd-4b73-bfe4-b5bd5ccabe24-0', usage_metadata={'input_tokens': 86, 'output_tokens': 65, 'total_tokens': 151, 'output_token_details': {'reasoning': 49}})

In [11]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Bonjour, comment ça va\u202f?'

In [12]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Bonjour, comment ça va\u202f?'

In [13]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [14]:
result=prompt.invoke({"language":"French","text":"Hello"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [11]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [15]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour'

In [18]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Marathi","text":"HI Meera, How are you?"})

'हाय मीरा, तू कशी आहेस?'

In [19]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Marathi","text":"My name is Vaibhav"})

'माझं नाव वैभव आहे.'

In [21]:
chain=prompt|model
chain.invoke({"language":"Marathi","text":"My name is Vaibhav"})

AIMessage(content='**Marathi:**  \nमाझे नाव वैभव आहे.', additional_kwargs={'reasoning_content': 'The user wants translation into Marathi: "My name is Vaibhav". Marathi translation: "माझं नाव वैभव आहे." Could also be "माझे नाव वैभव आहे." Both correct. Use proper Marathi script. Provide translation.'}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 86, 'total_tokens': 162, 'completion_time': 0.159892464, 'completion_tokens_details': {'reasoning_tokens': 53}, 'prompt_time': 0.003450394, 'prompt_tokens_details': None, 'queue_time': 0.047695565, 'total_time': 0.163342858}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_70d048ba3c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--5bf86934-ccd7-4a26-8797-295c85cdf09f-0', usage_metadata={'input_tokens': 86, 'output_tokens': 76, 'total_tokens': 162, 'output_token_details': {'reasoning': 53}})